In [35]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping

## Load Model

In [37]:
model = load_model("./Model/weights-improvement-15-1.1995-bigger.hdf5")

## Load Data

In [38]:
with open("./Files/notes.pkl", 'rb') as f:
    notes= pickle.load(f)

In [39]:
n_vocab = len(set(notes))

In [40]:
# ALl the unique elements in a sorted manner
pitchnames = sorted(set(item for item in notes))

In [41]:
# Mapping between note to int value
ele_to_int = dict((note, number) for number, note in enumerate(pitchnames))

# Predictions

In [42]:
sequence_length = 100
network_input = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i : i+sequence_length] # contains 100 values
    network_input.append([ele_to_int[ch] for ch in seq_in])

In [43]:
# Any random start index
start = np.random.randint(len(network_input) - 1)

# Mapping int_to_ele
int_to_ele = dict((num, ele) for num, ele in enumerate(pitchnames))

# Initial pattern 
pattern = network_input[start]
prediction_output = []

# generate 200 elements
for note_index in range(200):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1)) # convert into numpy desired shape 
    prediction_input = prediction_input/float(n_vocab) # normalise
    
    prediction =  model.predict(prediction_input, verbose=0)
    
    idx = np.argmax(prediction)
    result = int_to_ele[idx]
    prediction_output.append(result) 
    
    # Remove the first value, and append the recent value.. 
    # This way input is moving forward step-by-step with time..
    pattern.append(idx)
    pattern = pattern[1:]

In [44]:
print(prediction_output)

['B2', 'G3', '7+11', 'G3', 'F#2', 'G2', '7+0', 'G3', 'B2', 'G3', '7+11', 'G3', 'B2', 'A2', '7+0', 'A3', 'B2', 'G5', 'A5', 'D6', 'F#2', 'G3', 'B3', 'D6', 'B3', 'G3', 'B3', 'D6', 'B3', 'G3', 'B3', 'D6', 'B3', 'G3', 'B3', 'D6', 'B3', 'B5', 'B3', 'C3', 'G3', 'B5', 'C4', 'D4', 'B5', 'C3', 'C3', 'G3', 'B5', 'C4', 'B5', 'G3', 'B3', 'C3', 'G5', 'B5', 'C4', 'G5', 'B5', 'C4', 'G5', 'G5', 'B5', 'C4', 'G5', 'G3', 'D6', 'B3', 'C2', 'G3', 'B3', 'D4', 'B3', 'G3', 'B3', 'D6', 'B3', 'G3', 'B3', 'C3', 'D4', 'G5', 'B5', 'C4', 'G5', 'B5', 'C3', 'G5', 'G5', 'A5', 'D4', 'G5', 'G3', 'C3', 'C3', 'G5', 'B3', 'D6', 'D3', 'D3', 'B-2', 'B-2', 'F3', 'A2', 'B-2', 'C3', 'G5', 'B-2', 'C3', 'E-3', 'G2', 'B-2', 'C4', 'G5', 'D3', 'B-2', 'A3', 'G5', 'G5', 'E-3', 'B3', 'G2', 'G6', 'D6', 'B-2', '9+11', 'G3', 'D4', 'D6', 'G4', 'G5', 'D3', '7+0', 'D2', 'D4', '4+9', 'F2', 'C4', '9+0', 'C#5', 'C#5', 'C#5', 'A4', 'E4', 'C4', 'G4', 'G4', 'A4', 'D4', 'E4', 'D4', 'E4', 'D4', 'A3', 'A3', 'D4', 'D4', 'D4', 'E4', 'A3', 'D4', 'E4', 'E

## Create Midi File

In [45]:
offset = 0 # Time
output_notes = []

for pattern in prediction_output:
    
    # if the pattern is a chord
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        temp_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))  # create Note object for each note in the chord
            new_note.storedInstrument = instrument.Piano()
            temp_notes.append(new_note)
            
        
        new_chord = chord.Chord(temp_notes) # creates the chord() from the list of notes
        new_chord.offset = offset
        output_notes.append(new_chord)
    
    else:
            # if the pattern is a note
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
    offset += 0.5

In [46]:
# create a stream object from the generated notes
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp = "./Output Music/test_output.mid")

'./Output Music/test_output.mid'

In [47]:
midi_stream.show('midi')